# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Notebook Principal

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**http://fdelmazo.github.io/7506-Datos/**

**https://www.kaggle.com/datatouille2018/competitions**

Continuando la investigación sobre la empresa Trocafone realizada en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), se busca determinar la probabilidad de que un usuario del sitio realice una conversión en el período determinado.

Notebooks en orden de corrida y lectura:

0. [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) --> Familiarización con el set de datos y exploración de estos.

1. [Investigación Previa](https://fdelmazo.github.io/7506-Datos/TP2/investigacion.html) --> Con ayuda de lo trabajado en el TP1, se averiguan más cosas de las datos, en busqueda de que poder reutilizar.

2. [Creación de Dataframes](https://fdelmazo.github.io/7506-Datos/TP2/new-dataframes.html) --> Como parte del feature engineering, se crean dataframes nuevos con información de los productos del sitio y de como se accede a este (marcas, sistemas operativos, etc).

3. [Feature Engineering](https://fdelmazo.github.io/7506-Datos/TP2/feature-engineering.html) --> Busqueda de atributos de los usuarios de los cuales se busca predecir la conversión.

4. [Submission Framework](https://fdelmazo.github.io/7506-Datos/TP2/submission-framework.html) --> Pequeño framework para construir las postulaciones de labels. 

5. TP2 (este notebook)--> Teniendo todo en cuenta, usando los dataframes con todos los atributos buscados y encontrados, se definen y aplican los algoritmos de clasificación, se realizan los entrenamientos y posteriores predicciones de conversiones y finalmente se arman las postulaciones de labels.

In [ ]:
# Set-up inicial, se deja comentado para evitar instalarle módulos al usuario
## Primero, descargar los datasets de no tenerlos

# Antes de comenzar, setear las credenciales (usuario y token)

# 1. Visitar: https://www.kaggle.com/datatouille2018/account (con la cuenta que sea)
# 2. Tocar en Create New API Token
# 3. Guardar el archivo descargado en ~/.kaggle/kaggle.json

# !pip install kaggle # https://github.com/Kaggle/kaggle-api
# !kaggle competitions download -c trocafone -p data
# !unzip -q data/events_up_to_01062018.csv.zip -d data
# !rm data/events_up_to_01062018.csv.zip
# !ls data/

## Luego, descargar los módulos a utilizar a lo largo de todo el trabajo

# !pip install nbimporter
# !conda install -c conda-forge xgboost 

In [ ]:
import nbimporter # pip install nbimporter
import pandas as pd
import numpy as np
import calendar
import submission_framework as SF

In [ ]:
df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

display(df_users.head(), df_y.head())

## Algoritmos de Machine Learning

In [ ]:
seed = 42
test_size = 0.34

def full_framework_wrapper(model_name, model_function, columns=df_users.columns.tolist(), test_size=test_size, seed=seed, verbosity=0, all_in=False, submit=False,feature_importance_print=False):
    model_df_x = df_users[columns]
    model_df_y = df_y
    
    X, y = SF.fr1_extract_X_y(model_df_x, model_df_y)
    X_train, X_test, y_train, y_test = SF.fr2_train_test_split(X, y, seed, test_size)
    if all_in:
        model = model_function(X, y, seed)
    else:
        model = model_function(X_train, y_train, seed)
    accuracy = SF.fr4_accuracy_score(X_test, y_test, model, model_name)
    auc = SF.fr5_auc(X_test, y_test, model, model_name)
    X_to_predict, predictions = SF.fr6_extract_X_to_predict(model_df_x, model_df_y, model)
       
    if verbosity>=1: print('Model: {} - Columns: {} (AUC: {:.4f})'.format(model_name, columns, auc))  
    elif verbosity >=0: print('Model: {} - Accuracy: {:.4f} - AUC: {:.4f}'.format(model_name, accuracy, auc))  
    
    if feature_importance_print: 
        feature_importances = SF.fr7_print_information(model_df_x, model, X_to_predict, True)
        display(feature_importances)
            
    if submit:
        csv_name, submission = SF.fr8_to_csv(X_to_predict, predictions, model_name, auc)
        display(csv_name)
        return auc, csv_name
    
    return auc

---

### Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

def decision_tree(X_train, y_train, seed):
    tree = DecisionTreeClassifier(criterion='gini',
                              min_samples_leaf=5,
                              min_samples_split=5,
                              max_depth=3,
                              random_state=seed)

    tree.fit(X_train, y_train)
    return tree

full_framework_wrapper('decision_tree',decision_tree)

---

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def random_forest(X_train, y_train, seed):
    rf = RandomForestClassifier(n_estimators=20,
                           n_jobs=2,
                           min_samples_split=300,
                           random_state=seed,
                           class_weight='balanced')

    y_train.shape = y_train.shape[0]
    rf.fit(X_train, y_train)
    return rf

full_framework_wrapper('random_forest',random_forest)

In [ ]:
# NO DAR BOLA POR AHORA

# No era necesario hacer el entrenamiento final con el modelo
# entrenado con X_train. Conviene usar el set de entrenamiento
# entero.

# Lo de X_train y X_test es sólo (POR AHORA) para mostrar
# el accuracy_score choto que estamos teniendo

full_framework_wrapper('random_forest_allin',random_forest,all_in=True)


---

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model_name = 'knn'

rango = 5
def knn_gridsearch(x_train, y_train, x_test, y_test):
    y_train.shape = y_train.shape[0]
    k_max = (0,0)
    for k in range(1,100, rango):
        knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
        knn.fit(x_train, y_train)
        score = knn.score(x_test, y_test)*100
        if score > k_max[1]:
            k_max = (k,score)
        print("K: {}, {}".format(k, score))
    a = k_max[0]
    k_max = (0,0)
    for k in range(a - rango, a + rango):
        knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
        knn.fit(x_train, y_train)
        score = knn.score(x_test, y_test)*100
        if score > k_max[1]:
            k_max = (k,score)
        print("K: {}, {}".format(k, score))
    return k_max[0]

X, y = SF.fr1_extract_X_y(df_users, df_y)
X_train, X_test, y_train, y_test = SF.fr2_train_test_split(X, y, seed, test_size)    
K = knn_gridsearch(X_train, y_train, X_test, y_test)

def knn(X_train, y_train, seed, k=K):
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform', n_jobs=-1)
    y_train.shape = y_train.shape[0]
    knn.fit(X_train, y_train)
    return knn

full_framework_wrapper(f'KNN{K}',knn)

---

### XGBoost


In [ ]:
import xgboost as xgb #conda install -c conda-forge xgboost 

def xgboost(X_train, y_train, seed):
    xg_reg = xgb.XGBClassifier(objective='reg:linear',
                class_weight='balanced',
                colsample_bytree=0.3, learning_rate=0.1,
                min_child_weight=2,
                max_depth=5, alpha=10, n_estimators=20,
                random_state=seed)
    y_train.shape = y_train.shape[0]
    xg_reg.fit(X_train,y_train)
    return xg_reg

full_framework_wrapper('xgboost',xgboost)

In [ ]:
# # NO BORRAR ESTO !!!! ok, lo comento ;)

# total_predictions = 0
# tmp_seed = seed
# for i in range(50):
#     tmp_seed = tmp_seed + i
#     model = xgboost(X, y, tmp_seed)
#     print(tmp_seed)
#     X_to_predict, predictions = SF.fr6_extract_X_to_predict(fr_df, fr_df_y, model)
#     total_predictions = total_predictions + predictions

# predictions = total_predictions / 50

# """
# model = xgboost(X, y, seed)
# X_to_predict, predictions = SF.fr6_extract_X_to_predict(fr_df, fr_df_y, model)
# """

In [ ]:
# for i in range(len(predictions)):
#     if predictions[i] < 0:
#         print("WARNING: prediction[{}] = {}".format(i, predictions[i]))
#         predictions[i] = 0

---

## Encontrando el mejor submit

Usando Random Forest, el algoritmo más estable de los definidos (XGBoost es poco estable) encontramos que combinación de features es la mas favorable (con la métrica Area Under Curve).

Corremos todos los algoritmos definidos sobre esas combinaciones, en busqueda de su mejor combinación de hiper-parametros.

Finalmente, se corren todos los algoritmos en su mejor combinación contra todos los set de features definidos, en busqueda de la mejor fusión universal.

### Feature Selection

TODO: Pasar a un notebook nuevo

#### Cumulative Importance

Se parte de una lista de todos los features ordeandos segun importancia, y se genera una lista de listas acumulativa de esto. Es decir de `[a,b,c]` se pasa a `[ [a], [a,b], [a,b,c] ]`

Esto se hace porque se esta buscando el 'codo': Los features que hacen que incremente el AUC.

In [ ]:
X, y = SF.fr1_extract_X_y(df_users, df_y)
X_train, X_test, y_train, y_test = SF.fr2_train_test_split(X, y, seed, test_size)
model = random_forest(X_train, y_train, seed)
X_to_predict, predictions = SF.fr6_extract_X_to_predict(df_users, df_y, model)
feature_importances = SF.fr7_print_information(df_users, model, X_to_predict, True)

features_ordenados = feature_importances.index.tolist()
lista_progresiva_de_cols = [features_ordenados[:i] for i in range(1,len(features_ordenados))]

max_auc = full_framework_wrapper('random_forest',random_forest)
best_features_progresivo = features_ordenados
features_con_saltos_progresivo = []

for i, cols in enumerate(lista_progresiva_de_cols):
    print(f'\n\nIteración {i} de {len(lista_progresiva_de_cols)}\n\n')
    auc = full_framework_wrapper('random_forest',random_forest,columns=cols,verbosity=1)
    if auc > max_auc + 0.0001:
        max_auc = auc
        best_features_progresivo = cols
        features_con_saltos_progresivo.append(cols[-1])

Model: random_forest - Accuracy: 0.7908 - AUC: 0.8572


Iteración 0 de 61


Model: random_forest - Columns: ['has_checkout'] (AUC: 0.6854)


Iteración 1 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout'] (AUC: 0.7686)


Iteración 2 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout'] (AUC: 0.8383)


Iteración 3 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout'] (AUC: 0.8238)


Iteración 4 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5'] (AUC: 0.8438)


Iteración 5 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout'] (AUC: 0.8440)


Iteración 6 de 61


Model: random_forest - Columns: ['has_checkout', 

Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout', 'has_checkout_month_5', 'total_session_conversions', 'total_events', 'total_sessions', 'timestamp_last_event', 'total_checkouts', 'days_to_last_conversion', 'avg_events_per_session', 'total_checkouts_last_week', 'woy_last_checkout', 'dow_last_checkout', 'dom_last_event', 'has_checkout_last_week', 'total_checkouts_months_1_to_4', 'woy_last_conversion', 'days_to_last_viewed_product', 'timestamp_last_conversion', 'doy_last_event', 'has_checkout_months_1_to_4'] (AUC: 0.8573)


Iteración 25 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout', 'has_checkout_month_5', 'total_session_conversions', 'total_events', 'total_sessions', 'timestamp_last_event', 'total_checkouts', 'days_to_last_conversion', 'avg

Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout', 'has_checkout_month_5', 'total_session_conversions', 'total_events', 'total_sessions', 'timestamp_last_event', 'total_checkouts', 'days_to_last_conversion', 'avg_events_per_session', 'total_checkouts_last_week', 'woy_last_checkout', 'dow_last_checkout', 'dom_last_event', 'has_checkout_last_week', 'total_checkouts_months_1_to_4', 'woy_last_conversion', 'days_to_last_viewed_product', 'timestamp_last_conversion', 'doy_last_event', 'has_checkout_months_1_to_4', 'total_session_checkouts', 'days_to_last_event', 'has_conversion', 'doy_last_conversion', 'total_conversions_months_1_to_4', 'dow_last_event', 'total_checkouts_month_3', 'woy_last_event', 'total_conversions_last_week', 'dom_last_conversion', 'dow_last_conversion'] (AUC: 0.8580)


Iteración 36 de 61


Model: random_forest - Columns: ['has_checkout', 'days_to_last_che

Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout', 'has_checkout_month_5', 'total_session_conversions', 'total_events', 'total_sessions', 'timestamp_last_event', 'total_checkouts', 'days_to_last_conversion', 'avg_events_per_session', 'total_checkouts_last_week', 'woy_last_checkout', 'dow_last_checkout', 'dom_last_event', 'has_checkout_last_week', 'total_checkouts_months_1_to_4', 'woy_last_conversion', 'days_to_last_viewed_product', 'timestamp_last_conversion', 'doy_last_event', 'has_checkout_months_1_to_4', 'total_session_checkouts', 'days_to_last_event', 'has_conversion', 'doy_last_conversion', 'total_conversions_months_1_to_4', 'dow_last_event', 'total_checkouts_month_3', 'woy_last_event', 'total_conversions_last_week', 'dom_last_conversion', 'dow_last_conversion', 'amount_of_months_that_has_bought', 'has_checkout_month_4', 'total_checkouts_month_4', 'has_conversion_

Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout', 'has_checkout_month_5', 'total_session_conversions', 'total_events', 'total_sessions', 'timestamp_last_event', 'total_checkouts', 'days_to_last_conversion', 'avg_events_per_session', 'total_checkouts_last_week', 'woy_last_checkout', 'dow_last_checkout', 'dom_last_event', 'has_checkout_last_week', 'total_checkouts_months_1_to_4', 'woy_last_conversion', 'days_to_last_viewed_product', 'timestamp_last_conversion', 'doy_last_event', 'has_checkout_months_1_to_4', 'total_session_checkouts', 'days_to_last_event', 'has_conversion', 'doy_last_conversion', 'total_conversions_months_1_to_4', 'dow_last_event', 'total_checkouts_month_3', 'woy_last_event', 'total_conversions_last_week', 'dom_last_conversion', 'dow_last_conversion', 'amount_of_months_that_has_bought', 'has_checkout_month_4', 'total_checkouts_month_4', 'has_conversion_

Model: random_forest - Columns: ['has_checkout', 'days_to_last_checkout', 'doy_last_checkout', 'timestamp_last_checkout', 'total_checkouts_month_5', 'dom_last_checkout', 'has_checkout_month_5', 'total_session_conversions', 'total_events', 'total_sessions', 'timestamp_last_event', 'total_checkouts', 'days_to_last_conversion', 'avg_events_per_session', 'total_checkouts_last_week', 'woy_last_checkout', 'dow_last_checkout', 'dom_last_event', 'has_checkout_last_week', 'total_checkouts_months_1_to_4', 'woy_last_conversion', 'days_to_last_viewed_product', 'timestamp_last_conversion', 'doy_last_event', 'has_checkout_months_1_to_4', 'total_session_checkouts', 'days_to_last_event', 'has_conversion', 'doy_last_conversion', 'total_conversions_months_1_to_4', 'dow_last_event', 'total_checkouts_month_3', 'woy_last_event', 'total_conversions_last_week', 'dom_last_conversion', 'dow_last_conversion', 'amount_of_months_that_has_bought', 'has_checkout_month_4', 'total_checkouts_month_4', 'has_conversion_

In [ ]:
display(features_con_saltos_progresivo,best_features_progresivo)

['timestamp_last_conversion',
 'total_session_checkouts',
 'total_conversions_months_1_to_4',
 'has_checkout_month_3',
 'total_conversions_month_4',
 'has_checkout_month_2',
 'has_conversion_month_2']

['has_checkout',
 'days_to_last_checkout',
 'doy_last_checkout',
 'timestamp_last_checkout',
 'total_checkouts_month_5',
 'dom_last_checkout',
 'has_checkout_month_5',
 'total_session_conversions',
 'total_events',
 'total_sessions',
 'timestamp_last_event',
 'total_checkouts',
 'days_to_last_conversion',
 'avg_events_per_session',
 'total_checkouts_last_week',
 'woy_last_checkout',
 'dow_last_checkout',
 'dom_last_event',
 'has_checkout_last_week',
 'total_checkouts_months_1_to_4',
 'woy_last_conversion',
 'days_to_last_viewed_product',
 'timestamp_last_conversion',
 'doy_last_event',
 'has_checkout_months_1_to_4',
 'total_session_checkouts',
 'days_to_last_event',
 'has_conversion',
 'doy_last_conversion',
 'total_conversions_months_1_to_4',
 'dow_last_event',
 'total_checkouts_month_3',
 'woy_last_event',
 'total_conversions_last_week',
 'dom_last_conversion',
 'dow_last_conversion',
 'amount_of_months_that_has_bought',
 'has_checkout_month_4',
 'total_checkouts_month_4',
 'has_conv

####  Forward Selection

Se parte de una lista vacía y se van agregando todos los features uno por uno.

In [ ]:
features = df_users.columns.tolist()

cantidad_features = len(features)

max_auc = 0
features_no_usadas = features
features_ya_usadas = []
best_features_forward = []
features_con_saltos_forward = []

for i in range(cantidad_features+1):
    print(f'\n\nIteración {i} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_forward} (AUC: {max_auc:.4f}) \n\n')
    max_local_auc = 0
    for f in features_no_usadas:
        features_a_usar = features_ya_usadas + [f]
        auc = full_framework_wrapper('random_forest',random_forest,columns=features_a_usar,verbosity=1)
        if auc > max_local_auc + 0.0001:
            max_local_auc = auc
            feature_a_agregar = f
        
    features_ya_usadas += [feature_a_agregar]
    if features_no_usadas: features_no_usadas.remove(feature_a_agregar)
    
    if max_local_auc > max_auc + 0.0001:
        max_auc = max_local_auc
        best_features_forward = features_ya_usadas[:]
        features_con_saltos_forward.append(feature_a_agregar)



Iteración 0 de 62
Lo mejor al momento: [] (AUC: 0.0000) 


Model: random_forest - Columns: ['total_checkouts'] (AUC: 0.7551)
Model: random_forest - Columns: ['total_conversions'] (AUC: 0.5640)
Model: random_forest - Columns: ['total_events'] (AUC: 0.5510)
Model: random_forest - Columns: ['total_sessions'] (AUC: 0.6368)
Model: random_forest - Columns: ['total_session_checkouts'] (AUC: 0.5368)
Model: random_forest - Columns: ['total_session_conversions'] (AUC: 0.7571)
Model: random_forest - Columns: ['avg_events_per_session'] (AUC: 0.5627)
Model: random_forest - Columns: ['has_checkout'] (AUC: 0.6854)
Model: random_forest - Columns: ['has_conversion'] (AUC: 0.5620)
Model: random_forest - Columns: ['total_conversions_month_1'] (AUC: 0.5066)
Model: random_forest - Columns: ['total_checkouts_month_1'] (AUC: 0.5107)
Model: random_forest - Columns: ['has_checkout_month_1'] (AUC: 0.5107)
Model: random_forest - Columns: ['has_conversion_month_1'] (AUC: 0.5066)
Model: random_forest - Columns: 

Model: random_forest - Columns: ['doy_last_checkout', 'timestamp_last_event'] (AUC: 0.8327)
Model: random_forest - Columns: ['doy_last_checkout', 'timestamp_last_checkout'] (AUC: 0.8041)
Model: random_forest - Columns: ['doy_last_checkout', 'timestamp_last_conversion'] (AUC: 0.8439)
Model: random_forest - Columns: ['doy_last_checkout', 'timestamp_last_viewed_product'] (AUC: 0.8208)
Model: random_forest - Columns: ['doy_last_checkout', 'days_to_last_event'] (AUC: 0.8294)
Model: random_forest - Columns: ['doy_last_checkout', 'days_to_last_checkout'] (AUC: 0.8349)
Model: random_forest - Columns: ['doy_last_checkout', 'days_to_last_conversion'] (AUC: 0.8402)
Model: random_forest - Columns: ['doy_last_checkout', 'days_to_last_viewed_product'] (AUC: 0.8309)
Model: random_forest - Columns: ['doy_last_checkout', 'doy_last_event'] (AUC: 0.8309)
Model: random_forest - Columns: ['doy_last_checkout', 'dow_last_event'] (AUC: 0.8196)
Model: random_forest - Columns: ['doy_last_checkout', 'dom_last_ev

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'dom_last_conversion'] (AUC: 0.8544)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'woy_last_conversion'] (AUC: 0.8542)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'doy_last_viewed_product'] (AUC: 0.8503)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'dow_last_viewed_product'] (AUC: 0.8503)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'dom_last_viewed_product'] (AUC: 0.8503)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'woy_last_viewed_product'] (AUC: 0.8503)


Iteración 3 de 62
Lo mejor al momento: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session'] (AUC: 0.8573) 


Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_checkouts'] (AUC: 0.8616)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'av

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'woy_last_conversion'] (AUC: 0.8636)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'doy_last_viewed_product'] (AUC: 0.8598)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'dow_last_viewed_product'] (AUC: 0.8598)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'dom_last_viewed_product'] (AUC: 0.8598)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'woy_last_viewed_product'] (AUC: 0.8598)


Iteración 4 de 62
Lo mejor al momento: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts'] (AUC: 0.8643) 


Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'total_checkouts'] (AUC: 0.8614)


Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'dom_last_event'] (AUC: 0.8574)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_event'] (AUC: 0.8639)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'dow_last_checkout'] (AUC: 0.8584)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'dom_last_checkout'] (AUC: 0.8604)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_checkout'] (AUC: 0.8602)
Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'doy_last_conversion'] (AUC: 0.8610)
Model: random_forest - Columns: ['doy_last_c

...

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_event', 'dow_last_checkout', 'total_conversions_month_4', 'has_checkout_month_5', 'dow_last_conversion', 'has_conversion_last_week', 'timestamp_last_checkout', 'doy_last_conversion', 'has_checkout', 'has_checkout_month_1', 'total_checkouts', 'timestamp_last_event', 'total_checkouts_month_2', 'total_checkouts_month_1', 'timestamp_last_conversion', 'total_events', 'total_conversions', 'days_to_last_conversion', 'total_conversions_month_5', 'days_to_last_checkout', 'total_checkouts_months_1_to_4', 'woy_last_checkout', 'total_conversions_month_3', 'dom_last_event', 'has_conversion_month_3', 'total_conversions_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'has_checkout_last_week', 'dom_last_checkout', 'total_conversions_month_2', 'doy_last_viewed_product', 'total_conversions_month_1', 'has_conversion', 'total_checkouts_last_week

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_event', 'dow_last_checkout', 'total_conversions_month_4', 'has_checkout_month_5', 'dow_last_conversion', 'has_conversion_last_week', 'timestamp_last_checkout', 'doy_last_conversion', 'has_checkout', 'has_checkout_month_1', 'total_checkouts', 'timestamp_last_event', 'total_checkouts_month_2', 'total_checkouts_month_1', 'timestamp_last_conversion', 'total_events', 'total_conversions', 'days_to_last_conversion', 'total_conversions_month_5', 'days_to_last_checkout', 'total_checkouts_months_1_to_4', 'woy_last_checkout', 'total_conversions_month_3', 'dom_last_event', 'has_conversion_month_3', 'total_conversions_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'has_checkout_last_week', 'dom_last_checkout', 'total_conversions_month_2', 'doy_last_viewed_product', 'total_conversions_month_1', 'has_conversion', 'total_checkouts_last_week

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_event', 'dow_last_checkout', 'total_conversions_month_4', 'has_checkout_month_5', 'dow_last_conversion', 'has_conversion_last_week', 'timestamp_last_checkout', 'doy_last_conversion', 'has_checkout', 'has_checkout_month_1', 'total_checkouts', 'timestamp_last_event', 'total_checkouts_month_2', 'total_checkouts_month_1', 'timestamp_last_conversion', 'total_events', 'total_conversions', 'days_to_last_conversion', 'total_conversions_month_5', 'days_to_last_checkout', 'total_checkouts_months_1_to_4', 'woy_last_checkout', 'total_conversions_month_3', 'dom_last_event', 'has_conversion_month_3', 'total_conversions_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'has_checkout_last_week', 'dom_last_checkout', 'total_conversions_month_2', 'doy_last_viewed_product', 'total_conversions_month_1', 'has_conversion', 'total_checkouts_last_week

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_event', 'dow_last_checkout', 'total_conversions_month_4', 'has_checkout_month_5', 'dow_last_conversion', 'has_conversion_last_week', 'timestamp_last_checkout', 'doy_last_conversion', 'has_checkout', 'has_checkout_month_1', 'total_checkouts', 'timestamp_last_event', 'total_checkouts_month_2', 'total_checkouts_month_1', 'timestamp_last_conversion', 'total_events', 'total_conversions', 'days_to_last_conversion', 'total_conversions_month_5', 'days_to_last_checkout', 'total_checkouts_months_1_to_4', 'woy_last_checkout', 'total_conversions_month_3', 'dom_last_event', 'has_conversion_month_3', 'total_conversions_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'has_checkout_last_week', 'dom_last_checkout', 'total_conversions_month_2', 'doy_last_viewed_product', 'total_conversions_month_1', 'has_conversion', 'total_checkouts_last_week

Model: random_forest - Columns: ['doy_last_checkout', 'total_sessions', 'avg_events_per_session', 'total_session_checkouts', 'woy_last_event', 'dow_last_checkout', 'total_conversions_month_4', 'has_checkout_month_5', 'dow_last_conversion', 'has_conversion_last_week', 'timestamp_last_checkout', 'doy_last_conversion', 'has_checkout', 'has_checkout_month_1', 'total_checkouts', 'timestamp_last_event', 'total_checkouts_month_2', 'total_checkouts_month_1', 'timestamp_last_conversion', 'total_events', 'total_conversions', 'days_to_last_conversion', 'total_conversions_month_5', 'days_to_last_checkout', 'total_checkouts_months_1_to_4', 'woy_last_checkout', 'total_conversions_month_3', 'dom_last_event', 'has_conversion_month_3', 'total_conversions_last_week', 'days_to_last_viewed_product', 'timestamp_last_viewed_product', 'has_checkout_last_week', 'dom_last_checkout', 'total_conversions_month_2', 'doy_last_viewed_product', 'total_conversions_month_1', 'has_conversion', 'total_checkouts_last_week

In [ ]:
display(features_con_saltos_forward,best_features_forward)

['doy_last_checkout',
 'total_sessions',
 'avg_events_per_session',
 'total_session_checkouts',
 'timestamp_last_event']

['doy_last_checkout',
 'total_sessions',
 'avg_events_per_session',
 'total_session_checkouts',
 'woy_last_event',
 'dow_last_checkout',
 'total_conversions_month_4',
 'has_checkout_month_5',
 'dow_last_conversion',
 'has_conversion_last_week',
 'timestamp_last_checkout',
 'doy_last_conversion',
 'has_checkout',
 'has_checkout_month_1',
 'total_checkouts',
 'timestamp_last_event']

####  Backward Elimination

Se parte de una lista con todos los features y se van sacando uno por uno, en busqueda de cual hace que incremente un AUC una vez que se lo remueva.

In [ ]:
features = df_users.columns.tolist()
cantidad_features = len(features)

max_auc = full_framework_wrapper('random_forest',random_forest,verbosity=1)
max_local_auc = max_auc
best_features_backward = features
features_ya_usadas = features
features_con_saltos_backward = []

for i in range(cantidad_features+1):
    print(f'\n\nIteración {i} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_backward} (AUC: {max_auc:.4f}) \n\n')
    for f in features_ya_usadas:
        features_a_usar = features_ya_usadas[:]
        features_a_usar.remove(f)
        auc = full_framework_wrapper('random_forest',random_forest,columns=features_a_usar,verbosity=1)
        if auc > max_local_auc + 0.0001:
            max_local_auc = auc
            feature_a_borrar = f
            
    if feature_a_borrar in features_ya_usadas: features_ya_usadas.remove(feature_a_borrar)
        
    if max_local_auc > max_auc + 0.0001:
        max_auc = max_local_auc
        best_features_backward = features_ya_usadas[:]
        features_con_saltos_backward.append(feature_a_borrar)

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'am

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bou

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bou

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bo

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bou

...

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bo

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'amount_of_months_that_has_bought', 

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'amount_of_months_that_has_boug

Model: random_forest - Columns: ['total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkouts', 'total_session_conversions', 'avg_events_per_session', 'has_checkout', 'has_conversion', 'total_conversions_month_1', 'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3', 'has_checkout_month_3', 'has_conversion_month_3', 'total_conversions_month_4', 'total_checkouts_month_4', 'has_checkout_month_4', 'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4', 'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 'has_conversion_last_week', 'am

KeyboardInterrupt: 

In [ ]:
print('Tener en cuenta que en este caso, features_con_saltos_backward se refiere a aquellos features que al removerlos se ve un incremento. Es decir, son los features que menos sirven, correr el algoritmo solo sobre esto tendría resultados muy poco favorables.')
display(features_con_saltos_backward,best_features_backward)

Tener en cuenta que en este caso, features_con_saltos_backward se refiere a aquellos features que al removerlos se ve un incremento. Es decir, son los features que menos sirven, correr el algoritmo solo sobre esto tendría resultados muy poco favorables.


['doy_last_checkout']

['total_checkouts',
 'total_conversions',
 'total_events',
 'total_sessions',
 'total_session_checkouts',
 'total_session_conversions',
 'avg_events_per_session',
 'has_checkout',
 'has_conversion',
 'total_conversions_month_1',
 'total_checkouts_month_1',
 'has_checkout_month_1',
 'has_conversion_month_1',
 'total_conversions_month_2',
 'total_checkouts_month_2',
 'has_checkout_month_2',
 'has_conversion_month_2',
 'total_conversions_month_3',
 'total_checkouts_month_3',
 'has_checkout_month_3',
 'has_conversion_month_3',
 'total_conversions_month_4',
 'total_checkouts_month_4',
 'has_checkout_month_4',
 'has_conversion_month_4',
 'total_conversions_month_5',
 'total_checkouts_month_5',
 'has_checkout_month_5',
 'has_conversion_month_5',
 'total_conversions_months_1_to_4',
 'total_checkouts_months_1_to_4',
 'has_checkout_months_1_to_4',
 'has_conversion_months_1_to_4',
 'total_conversions_last_week',
 'total_checkouts_last_week',
 'has_checkout_last_week',
 'has_conversion_last_week',

In [ ]:
ss_cols_1 = ['total_checkouts_month_5',
'timestamp_last_checkout',
'timestamp_last_event',
'has_checkout_month_5',
'total_checkouts',
'days_to_last_event',
'total_checkouts_last_week',
'total_checkouts_months_1_to_4',
'total_conversions',
'total_session_conversions',
'total_events',
'total_sessions',
'avg_events_per_session',
'total_session_checkouts',
'has_checkout']

posibilidades_features = {
    'Best Cumulative Importance':best_features_progresivo,
    'Best Forward Selection':best_features_forward,
    'Best Backward Elimination':best_features_backward,
    'Leap Cumulative Importance':features_con_saltos_progresivo,
    'Leap Forward Selection':features_con_saltos_forward,
    'Selección a Mano': ss_cols_1
}

### Encontrando los mejores hiper-parametros para cada algoritmo

Con Grid Search se busca la mejor combinación de hiperparametros para cada algoritmo, haciendo un promedio de como performan contra cada uno de los `best_features`.

In [ ]:
# TODO: GRID SEARCH SOBRE TODOS LOS ALGORITMOS Y TODOS SUS HIPERPARAMETROS

In [ ]:
posibilidades_algoritmos = [('xgboost',xgboost),
                 ('random_forest',random_forest),
                 ('decision_tree',decision_tree),
                 (f'KNN{K}',knn)
]

### Encontrando la mejor combinación

Se corren todas las `best` version de cada algoritmo sobre todos los set de features importantes, en distintos ordenes

In [ ]:
CANTIDAD_ORDENES = 5

max_auc = 0
campeon_nombre = ''
campeon_algoritmo = None
campeon_forma = None
campeon_features = None

for nombre,algoritmo in posibilidades_algoritmos:
    for forma,features_seleccionadas in posibilidades_features.items():
        features_nuevos_ordenes = [np.random.permutation(features_seleccionadas) for x in range(CANTIDAD_ORDENES)]
        features_ordenes_posibles = features_nuevos_ordenes + [features_seleccionadas]
        for features_con_orden in features_ordenes_posibles:
            features = list(features_con_orden)
            auc = full_framework_wrapper(nombre,algoritmo,columns=features)
            if auc > max_auc:
                max_auc = auc
                campeon_nombre = nombre
                campeon_algoritmo = algoritmo
                campeon_forma = forma
                campeon_features = features
            
display(f"Mejor Apuesta: {campeon_nombre} ({max_auc:.2f} AUC) - Features: {campeon_forma}")
display(f"Orden Features: {campeon_features}")

## TODO: Encontrar mejor forma de ensamblar y de boostear

## Corrida Final

Se corre entrenando con X (y no X_train) el submit final.

In [ ]:
auc, csv_name = full_framework_wrapper(campeon_nombre+'_all_in',
                       campeon_algoritmo,
                       columns=campeon_features,
                       submit=True,
                       verbosity=1,
                       all_in=True)

In [ ]:
## Descomentar y submitear!
## Ojo, solo correr una vez!!!
%env FILE_NAME = {csv_name}
%env MESSAGE = {campeon_nombre}_all_in - {campeon_forma} - {campeon_features}

#!kaggle competitions submit -f $FILE_NAME -m "$MESSAGE" trocafone

print()
print('https://www.kaggle.com/c/trocafone/submissions?sortBy=date')
print('https://www.kaggle.com/c/trocafone/leaderboard')

---
---
## Algoritmos y Features

* PRECIOS

* Naive Bayes

* Perceprton

* SVM

* https://github.com/urielkelman/abracadata/tree/master/TP2

* https://github.com/MatiasReimondo/Datos


## Lista de cosas que se pueden hacer

* [ ] Identificar bias y varianza, ploteando error de set de entrenamiento y error de set de test en funcion de cantidad de datos en set de entrenamiento (mismo plot)

* [ ] Perturbar datos de entrada -> reducir overfitting

* [ ] Plotear AUC nuestra, AUC kaggle segun submission

* [ ] Catboost

* [ ] Reclamar 50 usd aws &#128544; 

* [ ] Clustering para nuevos features para entrenar